<a href="https://colab.research.google.com/github/jeonsion/2023-algorithm/blob/main/mini_proj_01_20191545_fianl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import plotly.io as pio
pio.renderers.default = "colab"

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [3]:
import tensorflow as tf

# Other imports
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split


import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import iplot

Using TensorFlow backend.


In [4]:
# 패션 MNIST 데이터셋 로드
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()


In [5]:
# 데이터 전처리
X_train = X_train_full / 255.0
X_test = X_test / 255.0

In [6]:
# 라벨 원-핫 인코딩
y_train = to_categorical(y_train_full, 10)
y_test = to_categorical(y_test, 10)


In [7]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000, 10)
(10000, 28, 28)
(10000, 10)


In [8]:
# 모델 정의
IMG_ROWS, IMG_COLS = 28, 28
NUM_CLASSES = 10


In [9]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

def build_model():
    model = Sequential()
    # Add convolution 2D
    model.add(Conv2D(32, kernel_size=(3, 3),
                    activation='relu',
                    kernel_initializer='he_normal',
                    input_shape=(IMG_ROWS, IMG_COLS, 1)))
    model.add(MaxPooling2D((2, 2)))
    # Add dropouts to the model
    model.add(Dropout(0.25))
    model.add(Conv2D(64,
                    kernel_size=(3, 3),
                    activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Add dropouts to the model
    model.add(Dropout(0.25))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    # Add dropouts to the model
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    # Add dropouts to the model
    model.add(Dropout(0.3))
    model.add(Dense(NUM_CLASSES, activation='softmax'))


    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer='adam',
                  metrics=['accuracy'])
    return model



model = build_model()


In [10]:
X_train = X_train.reshape(-1, 28, 28, 1)

In [11]:
k = 10
num_val = len(X_train) // k
num_epochs = 100
all_scores = []

for i in range(k):
  print('processing fold # ', i)
  X_val = X_train[i*num_val : (i+1) * num_val]
  y_val = y_train[i*num_val : (i+1) * num_val]
  X_train_part = np.concatenate(
    [X_train[:1*num_val],
     X_train[(i+1) * num_val:]],
    axis=0
  )
  y_train_part = np.concatenate(
    [y_train[:1*num_val],
     y_train[(i+1) * num_val:]],
    axis=0
  )





  model = build_model()
  history = model.fit(X_train_part, y_train_part,
            epochs=num_epochs,
            batch_size=256,
            verbose=1)

  val_loss, val_acc = model.evaluate(X_val, y_val, verbose=1)
  all_scores.append(val_acc)

processing fold #  0
Epoch 1/100
235/235 [==============================] - 2s 10ms/step - loss: 0.7967 - accuracy: 0.7023
Epoch 2/100
235/235 [==============================] - 2s 10ms/step - loss: 0.4996 - accuracy: 0.8152
Epoch 3/100
235/235 [==============================] - 2s 10ms/step - loss: 0.4283 - accuracy: 0.8428
Epoch 4/100
235/235 [==============================] - 2s 10ms/step - loss: 0.3865 - accuracy: 0.8580
Epoch 5/100
235/235 [==============================] - 3s 12ms/step - loss: 0.3555 - accuracy: 0.8704
Epoch 6/100
235/235 [==============================] - 2s 10ms/step - loss: 0.3357 - accuracy: 0.8781
Epoch 7/100
235/235 [==============================] - 2s 10ms/step - loss: 0.3168 - accuracy: 0.8842
Epoch 8/100
235/235 [==============================] - 2s 10ms/step - loss: 0.3044 - accuracy: 0.8884
Epoch 9/100
235/235 [==============================] - 3s 11ms/step - loss: 0.2914 - accuracy: 0.8939
Epoch 10/100
235/235 [==============================] - 2s 10

In [12]:
model.summary()


Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_30 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
dropout_40 (Dropout)         (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
dropout_41 (Dropout)         (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 3, 3, 128)       

In [13]:
def create_trace(x,y,ylabel,color):
    trace = go.Scatter(
        x = x,y = y,
        name=ylabel,
        marker=dict(color=color),
        mode = "markers+lines",
        text=x
    )
    return trace

def plot_accuracy_and_loss(history):
    hist = history.history
    acc = hist['accuracy'];;;;;
    val_acc = hist['val_accuracy']
    loss = hist['loss']
    val_loss = hist['val_loss']
    epochs = list(range(1,len(acc)+1))

    trace_ta = create_trace(epochs,acc,"Training accuracy", "Green")
    trace_va = create_trace(epochs,val_acc,"Validation accuracy", "Red")
    trace_tl = create_trace(epochs,loss,"Training loss", "Blue")
    trace_vl = create_trace(epochs,val_loss,"Validation loss", "Magenta")

    fig = make_subplots(rows=1,cols=2, subplot_titles=('Training and validation accuracy',
                                                          'Training and validation loss'))
    fig.append_trace(trace_ta,1,1)
    fig.append_trace(trace_va,1,1)
    fig.append_trace(trace_tl,1,2)
    fig.append_trace(trace_vl,1,2)
    fig['layout']['xaxis'].update(title = 'Epoch')

    iplot(fig)

plot_accuracy_and_loss(history)

KeyError: 'val_accuracy'